In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../../DATASETS_TRACTATS/TOT_fzv_1421_altitud.csv")

In [3]:
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [4]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
encoder = ce.OrdinalEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "ALTITUD", "VARIEDAD", "COLOR", "TIPO", "MODO"])
df = encoder.fit_transform(df)

In [6]:
df = df[df["CAMPAÑA"] != "22"].drop(columns ="SUPERFICIE")

In [7]:
X = df.drop(columns = ["PRODUCCION", "CAMPAÑA"])
y = df["PRODUCCION"]

In [9]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

def grid(X,y):
    gsc = GridSearchCV(
                estimator=xgb.XGBRegressor(),
                param_grid={"n_estimators": [100],
                            "learning_rate": [0.15],
                            "max_depth": [40],
                            "min_child_weight": [1],
                            "gamma":[0.1],
                            "colsample_bytree":[0.4],
                            "booster": ["dart"]},
                cv=5, scoring=["r2", "neg_root_mean_squared_error"], refit = "neg_root_mean_squared_error", verbose=0, n_jobs=-1)
    gsc.fit(X,y)
    
    return gsc

#### Random

In [10]:
model_random = grid(X,y)

In [11]:
model_random.best_score_

-6830.512036847834

In [12]:
pd.DataFrame(model_random.cv_results_).iloc[:, 14:25]

,split2_test_r2,split3_test_r2,split4_test_r2,mean_test_r2,std_test_r2,rank_test_r2,split0_test_neg_root_mean_squared_error,split1_test_neg_root_mean_squared_error,split2_test_neg_root_mean_squared_error,split3_test_neg_root_mean_squared_error,split4_test_neg_root_mean_squared_error
0,0.836691,0.768214,0.761937,0.7644,0.042812,1,-7690.801965,-7286.732658,-6225.945605,-6313.506978,-6635.572978


#### Testejant un any

In [13]:
years_to_test = ["14", "15", "16", "17", "18", "19", "20", "21"]
model_one_year_test = {}
for year in years_to_test:
    X_train = X[df["CAMPAÑA"] != year]
    y_train = y[df["CAMPAÑA"] != year]
    X_test = X[df["CAMPAÑA"] == year]
    y_test = y[df["CAMPAÑA"] == year]
    m_y = grid(X_train, y_train)
    score_y = m_y.score(X_test, y_test)
    model_one_year_test[year] = (m_y, score_y)

In [14]:
for year in years_to_test:
    print(year)
    print(pd.DataFrame(model_one_year_test[year][0].cv_results_).loc[:,["mean_test_r2", "mean_test_neg_root_mean_squared_error"]])
    print(model_one_year_test[year][1])

14
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.776433                           -6725.899079
-7809.276192239334
15
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.749264                           -6856.141688
-6963.390065893019
16
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.761905                           -6806.125482
-6976.970361800545
17
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.759096                           -7012.702358
-6029.396022524218
18
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.746322                           -6992.648321
-6384.2418594482015
19
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.738019                           -7346.034388
-4917.23807167744
20
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.762133                           -6883.241771
-7166.954023648184
21
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.755736   

#### Per anys

In [15]:
years_to_test2 = df["CAMPAÑA"].unique()
model_one_year = {}
for year in years_to_test2:
    X2 = X[df["CAMPAÑA"] == year]
    y2 = y[df["CAMPAÑA"] == year]
    m_y = grid(X2, y2)
    model_one_year[year] = m_y

In [16]:
for year in years_to_test2:
    print(year)
    print(pd.DataFrame(model_one_year[year].cv_results_).loc[:,["mean_test_r2", "mean_test_neg_root_mean_squared_error"]])

14
   mean_test_r2  mean_test_neg_root_mean_squared_error
0     -0.322994                          -13787.419996
15
   mean_test_r2  mean_test_neg_root_mean_squared_error
0     -0.036947                          -15389.601996
16
   mean_test_r2  mean_test_neg_root_mean_squared_error
0     -0.083617                          -14360.016722
17
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      -0.08037                           -11892.21996
18
   mean_test_r2  mean_test_neg_root_mean_squared_error
0     -0.143781                          -15170.827903
19
   mean_test_r2  mean_test_neg_root_mean_squared_error
0     -0.252232                          -12700.567018
20
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      -0.23537                          -13754.019118
21
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      -0.52561                          -14650.251709
